# **17次元の変数で表されるデータを2次元で表現する（次元削減）**

TaSi<sub>17</sub>クラスターの構造異性体95種(EQ0～EQ94)に関するエネルギー(Energy)とTa-Si核間距離(R1～R17)のデータがcsvファイルにまとめられています。

17次元の変数で表されるクラスター構造の情報を「主成分分析(PCA)」を用いて2次元に削減し、95個のデータを2次元平面にプロットしましょう。

TaSi.csvファイルはこちらから取得できます。

https://drive.google.com/file/d/1mE_pZ--mWrJ0mPoUaKY4uABO4NmxiJ7C/view?usp=sharing



In [ ]:
#必要そうなライブラリたちをインポート（コピペすべし）
from sklearn.datasets import make_blobs
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv

# **STEP1：データの読み込み**

In [ ]:
#csvファイルをアップロード
#左側の"ファイル"ボタンを押し、ファイルをdrag-and-dropしてもOK
#from google.colab import files
#files.upload()

In [ ]:
#csvファイルの中身を「df」に代入
## dfは"data frame"の略。
## pythonでは、data frameとarrayの2種類をよく使うので、どちらか混乱しないように、data frameにはdfと名前を付ける癖をつけるべし
df=pd.read_csv('TaSi17.csv')

# df_ML の中身を確認①
# 各列の名前やﾃﾞｰﾀの個数（ぬけがないか等々）をcheck
df.info()

In [ ]:
#dfの中身を確認②
# ﾃﾞｰﾀを上から4行だけ見てみる head(#) の#の数で表示する行数を指定
df.head(4)

# **STEP2：dfのデータを切り分け**

In [ ]:
# dfの中のR1～R17をXに代入したい
# つまり、EQ_ID(構造の名前)とEnergyを削除すれば良い
# 削除したdfを別名に代入する前に、正しくデータを削除できているか確認
df.drop(columns=['EQ_ID','Energy']).head(2)
# df.iloc[:,2:].head(2)  行・列の数を指定して削除することも可(2列目以降をすべて取り出すという意味)

In [ ]:
#核間距離(R1～R17)のみを取り出して、"df_dist"というデータフレームに代入
df_dist=df.drop(columns=['EQ_ID','Energy'])

#df_distの中身を確認
# ﾃﾞｰﾀを上から2行だけ見てみる
df_dist.head(2)

In [ ]:
# データの分散具合を見てみる：その１
df_AllData = df.drop(columns='EQ_ID')  #構造名以外の全データを取り出す

pd.plotting.scatter_matrix(df_AllData, figsize=(20, 20), hist_kwds={'bins':15},
                           marker=('o'), s=8, alpha=.5)
plt.show()

In [ ]:
# データの分散具合を見る：その２：相関行列
correlation_coefficients = df_AllData.corr()  # 相関行列の計算
# 相関行列のヒートマップ (相関係数の値あり)
plt.rcParams['font.size'] = 9
plt.figure(figsize=(18, 12))  # この段階で画像のサイズを指定する
sns.heatmap(correlation_coefficients, vmax=1, vmin=-1, cmap='seismic', square=True, annot=True, xticklabels=1, yticklabels=1)
plt.xlim([0, correlation_coefficients.shape[0]])
plt.show()

# **STEP3：PCAによる次元圧縮**

In [ ]:
from sklearn.decomposition import PCA  # PCA用ライブラリをインポート
# k次元に圧縮したい場合、n_components = k と指定する。
pca = PCA(n_components=2, random_state=10)
pca.fit(df_dist)

In [ ]:
# 寄与率（表現できる情報量の割合）を計算する
contribution_ratios = pd.DataFrame(pca.explained_variance_ratio_)
contribution_ratios

In [ ]:
#ローディングベクトルを表示する
#第一主成分は(0.672)×(R17) + (0.667)×(R16) + (0.312)×(R15) +…   で構成されていることが読み取れる
loadings = pd.DataFrame(pca.components_.T, index=df_dist.columns)
loadings

In [ ]:
# score(主成分得点)は各サンプルが各主成分軸上のどの座標に位置するかを表す値
score = pd.DataFrame(pca.transform(df_dist), index=df_dist.index)
score

In [ ]:
# 第 1 主成分と第 2 主成分の散布図 (energy の値でサンプルに色付け)
plt.scatter(score.iloc[:, 0], score.iloc[:, 1], c=df['Energy'], cmap=plt.get_cmap('jet'))
clb = plt.colorbar()
clb.set_label('Energy', labelpad=-20, y=1.1, rotation=0)
#plt.xlim(-1.5, 2.5)
#plt.ylim(-1.0, 1.0)
plt.xlabel('comp_0')
plt.ylabel('comp_1')
plt.grid()
plt.show()

In [ ]:
num = len(df_dist)  # 可視化するデータ数(＝行数)を指定
plt.scatter(score.iloc[:,0], score.iloc[:,1])
plt.rcParams["font.size"] = 8
# プロットしたデータにサンプル名をラベリング
for i in range(num):
    plt.text(score.iloc[i,0], score.iloc[i,1], score.index[i], horizontalalignment="center", verticalalignment="bottom")
#plt.xlim(-1.5, 2.5)
#plt.ylim(-1.0, 1.0)
plt.xlabel("comp_0")
plt.ylabel("comp_1")
plt.grid()
plt.show()

In [ ]:
#おまけ
#せっかくなので、comp_0, comp_1を説明変数とした重回帰分析もしてみる（主成分回帰（Principal Component Regression, PCR）)
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
model = LinearRegression()
X = score.iloc[:,:]
Y = df['Energy']
model.fit(X, Y)

# 決定係数
print("R^2=",model.score(X,Y))
# 回帰係数
print(model.coef_)